In [1]:
from PIL import Image, ImageDraw, ImageFont
import os
import json
from skimage.metrics import structural_similarity as ssim
import numpy as np

In [8]:
dataset_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\sewfactory'
save_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\Paula_dataset'

def get_70_70_images(name):
    good_dresses = []
    images = []
    for folder_dress in os.listdir(dataset_path):
        if folder_dress.startswith(name):
                try:
                    image = Image.open(os.path.join(dataset_path, folder_dress, 'static/static_0_30.png'))
                    # resize to 20x20
                    image = image.resize((70, 70))
                    # if pixel is not white, make it black
                    for i in range(image.size[0]):
                        for j in range(image.size[1]):
                            if image.getpixel((i, j)) != (255, 255, 255):
                                image.putpixel((i, j), (0, 0, 0))
                    image = image.convert('L')
                    image_np = np.array(image)
                    images.append(image_np)
                    image.save(os.path.join(save_path, folder_dress+'.png'))
                    good_dresses.append(folder_dress)
                except:
                    print('error on ', folder_dress)
                    continue

    return images, good_dresses

images, good_dresses = get_70_70_images('dress')


error on  dress_sleeveless_7716FF7QWP


In [70]:
def find_uniques(unique_images):
    counter=0
    threshold = 0.98
    n = len(unique_images)
    similar_pairs_unique = []
    for i in range(n):
        index_i = good_dresses.index(unique_images[i])
        for j in range(i + 1, n):
            index_j =  good_dresses.index(unique_images[j])
            sim = ssim(images[index_i], images[index_j])
            if sim > threshold:
                #print(good_dresses[i], good_dresses[j], sim)
                counter = counter + 1
                similar_pairs_unique.append((unique_images[i], unique_images[j], sim))
                
        if i%100 == 0:
            print("indice:", i, "pares encontrados:", counter)

    unique_images = []
    for i in range(len(similar_pairs_unique)):
        unique_images.append(similar_pairs_unique[i][0])

    unique_images = np.unique(unique_images)
    print("----------------------------\n"
          "----------------------------\n"
          "----------------------------\n")
    print(len(unique_images))
    return unique_images



unique_images = good_dresses.copy()
unique_images = find_uniques(unique_images)

indice: 0 pares encontrados: 17
indice: 100 pares encontrados: 206
indice: 200 pares encontrados: 319
indice: 300 pares encontrados: 461
indice: 400 pares encontrados: 582
indice: 500 pares encontrados: 697
indice: 600 pares encontrados: 790
indice: 700 pares encontrados: 886
indice: 800 pares encontrados: 1040
indice: 900 pares encontrados: 1121
indice: 1000 pares encontrados: 1237
indice: 1100 pares encontrados: 1335
indice: 1200 pares encontrados: 1391
indice: 1300 pares encontrados: 1431
indice: 1400 pares encontrados: 1487
indice: 1500 pares encontrados: 1526
indice: 1600 pares encontrados: 1551
indice: 1700 pares encontrados: 1597
indice: 1800 pares encontrados: 1631
indice: 1900 pares encontrados: 1656
indice: 2000 pares encontrados: 1699
indice: 2100 pares encontrados: 1724
indice: 2200 pares encontrados: 1737
indice: 2300 pares encontrados: 1750
indice: 2400 pares encontrados: 1758
indice: 2500 pares encontrados: 1760
----------------------------
----------------------------
-

In [68]:
len(unique_images)

167

In [74]:
unique_images_v2 = find_uniques(unique_images_v2)

indice: 0 pares encontrados: 9
indice: 100 pares encontrados: 402
indice: 200 pares encontrados: 665
----------------------------
----------------------------
----------------------------

167


In [53]:
def save_unique_images(unique_images): 
    save_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\Paula_dataset_unique'

    for folder_dress in unique_images:
        image = Image.open(os.path.join(dataset_path, folder_dress, 'static/static_0_30.png'))    
        image = image.convert('L')
        image.save(os.path.join(save_path, folder_dress+'.png'))

In [75]:
save_unique_images(unique_images_v2)

In [85]:
with open('unique_atributes.txt', 'r') as file_unique:
    for line in file_unique:
        print(line)

Tamaño de manga: 

    - Ancha

    - Estrecha



Largo de manga:

    - Sisa

    - Corta

    - Al hombro

  

Cuello: 

    - En V

    - Redondo



Largo de falda: 

    - Larga 

    - Corta

    - Midi



Ancho de falda:

    - Ancha

    - Pegada



In [91]:
new_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\Paula_dataset_unique'
import shutil



# open txt file unique_atributes.txt

for file in os.listdir(new_path):
    name = file.split('.')[0]
    shutil.copyfile('unique_atributes.txt', os.path.join(new_path, name+'_atributes.txt'))




Tamaño de manga
1. Ancha
2. Estrecha
['Tamaño de manga: Ancha\n']
Largo de manga
1. Sisa
2. Corta
3. Al hombro
['Tamaño de manga: Ancha\n', 'Largo de manga: Corta\n']
Cuello
1. En V
2. Redondo
['Tamaño de manga: Ancha\n', 'Largo de manga: Corta\n', 'Cuello: Redondo\n']
Largo de falda
1. Larga
2. Corta
3. Midi
['Tamaño de manga: Ancha\n', 'Largo de manga: Corta\n', 'Cuello: Redondo\n', 'Largo de falda: Larga\n']
Ancho de falda
1. Ancha
2. Pegada
['Tamaño de manga: Ancha\n', 'Largo de manga: Corta\n', 'Cuello: Redondo\n', 'Largo de falda: Larga\n', 'Ancho de falda: Pegada\n']
Tamaño de manga
1. Ancha
2. Estrecha


KeyboardInterrupt: Interrupted by user